In [1]:
import requests
import numpy as np
import pandas as pd
import json
from dotenv import load_dotenv
load_dotenv() 
import os
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/sentence-t5-base")

def get_director(id):
    headers = {
        "accept": "application/json",
        "Authorization": "Bearer " + os.getenv("TMDB_API_KEY"),
    }
    url_credits = f"https://api.themoviedb.org/3/movie/{id}/credits?language=en-US"
    response = requests.get(url_credits, headers=headers)
    data_credits = response.json()

    for d in data_credits["crew"]:
        if d["job"] == "Director":
            return d["name"]
        
def get_film(id):
    url_movie = f"https://api.themoviedb.org/3/movie/{id}?language=en-US"
    headers = {
        "accept": "application/json",
        "Authorization": (
            ("Bearer "+os.getenv("TMDB_API_KEY"))
        ),
    }
    response = requests.get(url_movie, headers=headers)
    data = response.json()
    movies_keys_to_remove = [
        "backdrop_path",
        "belongs_to_collection",
        "homepage",
        "poster_path",
        "production_companies",
        "production_countries",
        "spoken_languages",
        "status",
        "tagline",
        "video",
    ]
    for k in movies_keys_to_remove:
        if k in data:
            data.pop(k)

    url_credits = f"https://api.themoviedb.org/3/movie/{id}/credits?language=en-US"
    response = requests.get(url_credits, headers=headers)
    data_credits = response.json()
    if "cast" not in data_credits:
        top_actors = []
    else:
        top_actors = data_credits["cast"][0:2]
    actors_keys_to_remove = [
        "adult",
        "known_for_department",
        "original_name",
        "cast_id",
        "credit_id",
        "order",
        "profile_path",
    ]

    for t in top_actors:
        for k in actors_keys_to_remove:
            if k in t:
                t.pop(k)

    data["actors"] = top_actors

    embeddings = model.encode(data["overview"] if len(data["overview"]) > 0 else data["title"])
    data["overview_embedding"] = embeddings.tolist()
    data["director"] = get_director(id)
    return data

import tqdm
def sample(ids, file_path):
    films = {}
    for id in tqdm.tqdm(ids):
        try:
            films[id] = get_film(id)
        except:
            print(f"Error with {id}")
    with open(file_path, "w") as outfile:
        json.dump(films, outfile)

/home/gio/anaconda3/envs/MPR/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# movies for 1 shot prompts
sample([269149, 953, 116977,157336, 313369, 216015, 245891, 155], './sample_prompt_1.json')

100%|██████████| 8/8 [00:02<00:00,  3.98it/s]
